In [1]:
import numpy as np
import pyedflib
from matplotlib import pyplot as plt
from nitime import utils
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries
from nitime.viz import plot_tseries
import csv
import pywt
import scipy.stats as sp
from scipy import signal
from spectrum import *
from os import listdir
from os.path import isfile, join
from wyrm import processing as proc
from wyrm.types import Data
from wyrm.processing import select_channels
from wyrm.processing import swapaxes

In [2]:
data_files_location = '4_class_data'
files1  = [f for f in listdir(data_files_location)]

In [3]:
files1

['k6b_markers.csv',
 'l1b_data.csv',
 'l1b_markers.csv',
 'k3b_markers.csv',
 'k3b_data.csv',
 'k6b_data.csv']

In [4]:
subjects = ['k3b','k6b','l1b']
location = data_files_location + '/' + subjects[1] + '_markers.csv'
mark = np.genfromtxt (location, delimiter=",")

In [5]:
ca = [1,2,3,4,5]

In [6]:
ca[0:4]

[1, 2, 3, 4]

# Wavelet-Domain Features

In [7]:
def wavelet_features(epoch,channels):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    for i in range(channels):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(channels):   
        cA_mean.append(np.mean(cA_values[x]))
        cA_std.append(abs(np.std(cA_values[x])))
        cA_Energy.append(abs(np.sum(np.square(cA_values[x]))))
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        cD_std.append(abs(np.std(cD_values[x])))
        cD_Energy.append(abs(np.sum(np.square(cD_values[x]))))
        Entropy_D.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        Entropy_A.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))) 
    return cA_std[25],cA_std[26],cA_std[27],cA_std[28],cA_std[29],cA_std[30],cA_std[31],cA_std[32],cA_std[33],cA_std[34],cA_std[35],cD_std[25],cD_std[26],cD_std[27],cD_std[28],cD_std[29],cD_std[30],cD_std[31],cD_std[32],cD_std[33],cD_std[34],cD_std[35],cA_Energy[25],cA_Energy[26],cA_Energy[27],cA_Energy[28],cA_Energy[29],cA_Energy[30],cA_Energy[31],cA_Energy[32],cA_Energy[33],cA_Energy[34],cA_Energy[35],cD_Energy[25],cD_Energy[26],cD_Energy[27],cD_Energy[28],cD_Energy[29],cD_Energy[30],cD_Energy[31],cD_Energy[32],cD_Energy[33],cD_Energy[34],cD_Energy[35],Entropy_D[25],Entropy_D[26],Entropy_D[27],Entropy_D[28],Entropy_D[29],Entropy_D[30],Entropy_D[31],Entropy_D[32],Entropy_D[33],Entropy_D[34],Entropy_D[35],Entropy_A[25],Entropy_A[26],Entropy_A[27],Entropy_A[28],Entropy_A[29],Entropy_A[30],Entropy_A[31],Entropy_A[32],Entropy_A[33],Entropy_A[34],Entropy_A[35],np.sum(cA_mean)/channels,np.sum(cA_std)/channels,np.sum(cD_mean)/channels,np.sum(cD_std)/channels,np.sum(cA_Energy)/channels,np.sum(cD_Energy)/channels,np.sum(Entropy_A)/channels,np.sum(Entropy_D)/channels

# Frequency domain features

In [8]:
from scipy import signal

def maxPwelch(data_win,Fs,channels):
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([channels,(len(BandF)-1)]);
    feature = []
    for j in range(60):
        f,Psd = signal.welch(epoch[j,:], 250)
        for i in range(len(f)):
            if f[i] > 30:
                feature.append(sum(Psd[0:i])/i)
            break
    return feature[25:36] 

# Autoregressive model parameters

In [9]:
def autogressiveModelParametersBurg(labels,channels):
    feature = []
    feature1 = []
    model_order = 6
    for i in range(channels):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for j in range(channels):
        for i in range(model_order):
            feature1.append(feature[j][i])

    return feature1

In [12]:
def find_variance(filtered_epoch,channels):
    return abs(np.std(epoch[25])),abs(np.std(epoch[26])),abs(np.std(epoch[27])),abs(np.std(epoch[28])),abs(np.std(epoch[29])),abs(np.std(epoch[30])),abs(np.std(epoch[31])),abs(np.std(epoch[32])),abs(np.std(epoch[33])),abs(np.std(epoch[34])),abs(np.std(epoch[35]))

In [13]:
csvfile = "Features/features1.csv"
feature_names = ['Activity','Mobility','Complexity','Kurtosis','2nd Difference Mean','2nd Difference Max','Coeffiecient of Variation','Skewness','1st Difference Mean','1st Difference Max',
          'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Energy',
          'Wavelet Approximate Entropy','Wavelet Detailed Entropy','Variance','Mean of Vertex to Vertex Slope','FFT Delta MaxPower','FFT Theta MaxPower','FFT Alpha MaxPower','FFT Beta MaxPower'
          ]
with open(csvfile, "a") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(feature_names) 

In [14]:
with open(csvfile, "a") as output:
    writer = csv.writer(output, lineterminator='\n') 
    for subject in subjects:
        
        location_data  = data_files_location + '/' + subject + '_data.csv'
        location_markers = data_files_location + '/' + subject + '_markers.csv'
        
        data = np.genfromtxt (location_data, delimiter=",")
        markers = np.genfromtxt (location_markers, delimiter = ",")
        
        for each_marker in markers:
            starting_point = each_marker[0]
            target = each_marker[1]
            
            if target>0 and target<=4:
                sound_stimuli_begins = starting_point + (2*250)
                visual_stimuli_begins = sound_stimuli_begins + 250
                end_of_visual = visual_stimuli_begins + 1000
                epoch = data[visual_stimuli_begins:end_of_visual,:]
                
                epoch = np.array(epoch)
                epoch = np.transpose(epoch)
                features = []
                channels = 60
                
                #Band-pass filtered
                (b, a) = signal.butter(3, np.array([8, 30])/ 100.00, 'bandpass')
                filtered_epoch = signal.lfilter(b, a, epoch, 1)
                
                feature_list = wavelet_features(filtered_epoch,channels)
                for feat in feature_list:
                    features.append(feat)
                    
                feature_list  =  maxPwelch(filtered_epoch,100,channels)
                for feat in feature_list:
                    features.append(feat)
                    
                    
                #Autoregressive model Coefficients
                feature_list = autogressiveModelParametersBurg(filtered_epoch,channels)
                for feat in feature_list:
                    features.append(feat.real)
                    
                feature_list = find_variance(filtered_epoch,channels)
                
                for feat in feature_list:
                    features.append(feat)
                    
                    
                features.append(target);
                writer.writerow(features)  

/home/joy/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
